In [1]:
import pandas as pd
import numpy as np
import pickle 
import statsmodels.api as sm
import matplotlib.pyplot as plt

%matplotlib inline

First we'll read in the cleaned album files -- 8,907 in total. We have the following columns:
* Band name
* Album name
* Number of songs on the album
* Release year
* Editor review score
* Number of user reviewers
* Average user review score
* Album metalness score
* Song titles metalness score
* Band name metalness score

Each metalness score is calculated in three different ways across words in the item: mean metalness score, sum of metalness scores, and the sum of metalness scores divided by the number of words in the item plus one.

This information was scraped from MetalStorm.net in October 2018. I have not made the data available (nor the code used to scrape and clean it) in this repo for copyright reasons.

Metalness scores are calculated using Iain Barr's metalness index (https://github.com/ijmbarr/pythonic-metal)

In [2]:
model_df = pd.read_pickle("metal_for_modeling.pkl")

# Metalness ratings eyeball check

As a reasonableness check, here are the 10 most and least metal album titles for each of the three methods I used to calculate album title metalness.

First, the sum of metalness scores:

In [3]:
print(model_df.sort_values(by='album_metalness_sum', ascending=False).album.head(10))
print(model_df.sort_values(by='album_metalness_sum').album.head(10))

2045     As Blood Rains From The Sky...We Walk The Path...
14522                  Time Will Die And Love Will Bury It
2133                               Rage Of The Blood Beast
7014                      Let Mortal Heroes Sing Your Fame
10755                 Hear Nothing See Nothing Say Nothing
2638                   The Wind Blows Witches From The Sky
10337                 Woods 5: Grey Skies & Electric Light
157                                     Scream Bloody Gore
4814                      Beyond Darkness And Hell We Come
9152                                  Kiss The Sun Goodbye
Name: album, dtype: object
3520     Doomsday For The Deceiver 20th Anniversary Spe...
4942                             20 Year Anniversary Party
13832                              Nato Per Ragioni Ignote
2937     Medical Malpractioners Of Pathological Pervers...
9124                           Prolonged Active Antagonism
14643                                The Density Parameter
4009                         

Next, the mean metalness scores:

In [4]:
print(model_df.sort_values(by='album_metalness_mean', ascending=False).album.head(10))
print(model_df.sort_values(by='album_metalness_mean').album.head(10))

11115                   Burn
8063                    Burn
517      As The Palaces Burn
11545            In My Veins
11271               Eternity
4100                Eternity
801             All Eternity
724                 Eternity
1249                Eternity
6387                 Breathe
Name: album, dtype: object
5755     Crumb's Crunchy Delights Organization
7185                             Carte Blanche
13165                     District Of Dystopia
4742                Blackened Theological Tome
1457                         666 International
891                                  Section X
3079                 Chainsaw Cesarean Section
2031                       Martie Peters Group
2952      Arrangements For Fulminating Vective
13486                                   Alaska
Name: album, dtype: object


Finally, our compromise version: the sum of the metalness scores divided by the number of words plus one:

In [5]:
print(model_df.sort_values(by='album_metalness_sum_div_len2', ascending=False).album.head(10))
print(model_df.sort_values(by='album_metalness_sum_div_len2').album.head(10))

2133     Rage Of The Blood Beast
7803            Sword Of Revenge
10518            Tears And Ashes
2857              Burn Me Wicked
4101              Bury The Ashes
9250      From Chaos To Eternity
10504      The Reign Of Darkness
13818     Trapped In The Shadows
157           Scream Bloody Gore
730            Ashes Of The Wake
Name: album, dtype: object
5755                 Crumb's Crunchy Delights Organization
3520     Doomsday For The Deceiver 20th Anniversary Spe...
13832                              Nato Per Ragioni Ignote
7185                                         Carte Blanche
13165                                 District Of Dystopia
4742                            Blackened Theological Tome
1457                                     666 International
3079                             Chainsaw Cesarean Section
891                                              Section X
2031                                   Martie Peters Group
Name: album, dtype: object


All of these versions look pretty good to me, although the compromise version looks the best based on the eye test.

We'll use all three in our modeling to see if any version predicts better than the others.

The above was fun but we don't need the band and album names for modeling.

In [6]:
model_df.drop(columns = ['band', 'album'], inplace = True)
model_df.dtypes

num_songs                           float64
release_year                        float64
review_score_num                    float64
user_review_num                     float64
user_review_score_num               float64
album_metalness_sum                 float64
album_metalness_mean                float64
album_metalness_sum_div_len2        float64
songtitle_metalness_sum             float64
songtitle_metalness_mean            float64
songtitle_metalness_sum_div_len2    float64
bandname_metalness_sum              float64
bandname_metalness_mean             float64
bandname_metalness_sum_div_len2     float64
dtype: object

Let's go ahead and standardize all of the numeric variables for modeling flexibility. 

In [7]:
for col in model_df.columns:
    col_zscore = col + '_zscore'
    model_df[col_zscore] = (model_df[col] - model_df[col].mean())/model_df[col].std()

model_df.drop(columns = ['num_songs', 'release_year', 'review_score_num', 'user_review_num', 'user_review_score_num', 
                        'album_metalness_sum', 'album_metalness_mean', 'album_metalness_sum_div_len2', 
                        'songtitle_metalness_sum', 'songtitle_metalness_mean', 'songtitle_metalness_sum_div_len2',
                        'bandname_metalness_sum', 'bandname_metalness_mean', 'bandname_metalness_sum_div_len2'], 
              inplace = True)

model_df.head(10)

,num_songs_zscore,release_year_zscore,review_score_num_zscore,user_review_num_zscore,user_review_score_num_zscore,album_metalness_sum_zscore,album_metalness_mean_zscore,album_metalness_sum_div_len2_zscore,songtitle_metalness_sum_zscore,songtitle_metalness_mean_zscore,songtitle_metalness_sum_div_len2_zscore,bandname_metalness_sum_zscore,bandname_metalness_mean_zscore,bandname_metalness_sum_div_len2_zscore
1,0.264812,-0.668986,-0.312380,3.836850,0.891848,-1.232209,-1.131751,-1.227162,1.547429,1.028148,1.234784,-0.355976,-0.355177,-0.362025
2,1.682120,-0.668986,0.519286,0.785471,0.879380,0.471374,0.313905,0.420432,2.094772,0.320208,0.492251,1.095925,0.351052,0.578906
3,0.264812,-1.407541,0.269787,-0.235790,0.480412,0.414454,0.265602,0.365382,0.853938,0.880424,1.028438,-0.355976,-0.355177,-0.362025
4,0.264812,-1.112119,0.602453,-0.219252,0.218588,0.696469,0.504920,0.638129,0.656205,0.495786,0.616554,-0.355976,-0.355177,-0.362025
5,0.264812,-0.668986,0.768786,-0.153097,0.779638,0.832484,0.620341,0.769673,0.842641,0.282350,0.405901,-0.355976,-0.355177,-0.362025
7,-0.018649,-0.668986,0.602453,-0.074538,0.081443,1.162869,0.396314,0.658065,-1.403473,-2.124611,-2.155921,-0.355976,-0.355177,-0.362025
9,3.666351,-1.112119,0.935120,-0.004249,0.941719,-1.608704,-1.451244,-1.591283,3.771442,0.251359,0.440831,-0.984158,-0.813515,-0.904830
10,0.264812,-0.964408,1.766786,-0.099346,0.891848,-1.608704,-1.451244,-1.591283,1.036765,0.718805,0.876841,-0.984158,-0.813515,-0.904830
11,0.264812,-0.816697,0.935120,-0.306079,0.305863,0.175108,0.737455,0.518524,0.410804,0.243198,0.335908,-0.355976,-0.355177,-0.362025
12,-0.302111,-1.702963,1.350953,0.690374,0.131314,-0.620277,-0.612468,-0.635342,0.419735,0.770333,0.869283,-0.355976,-0.355177,-0.362025


# Modeling

Let's start with a correlation matrix using our standardized variables. Our two potential outputs are review_score_num_zscore and user_review_score_num_zscore, so let's first check for missing values in each column. 

We have 712 albums with no editor review score and 1,544 with no user review score. We're going to drop the rows with missing user review scores since they are also missing the number of user reviewers, one of our input variables.

We'll keep the rows with missing editor review scores for the time being, although we will have to drop them from any model we create that attempts to predict editor review scores instead of user review scores.

There's not much here in the correlation matrix ...

In [8]:
print("Missing editor review scores: ", model_df['review_score_num_zscore'].isnull().sum())
print("Missing user review scores: ", model_df['user_review_score_num_zscore'].isnull().sum())

model_df.dropna(subset = ['user_review_score_num_zscore'], inplace=True)
print("Missing user review scores after drops: ", model_df['user_review_num_zscore'].isnull().sum())
print("Number of albums remaining: ", model_df.num_songs_zscore.count())

model_df.corr()

Missing editor review scores:  712
Missing user review scores:  1544
Missing user review scores after drops:  0
Number of albums remaining:  7363


,num_songs_zscore,release_year_zscore,review_score_num_zscore,user_review_num_zscore,user_review_score_num_zscore,album_metalness_sum_zscore,album_metalness_mean_zscore,album_metalness_sum_div_len2_zscore,songtitle_metalness_sum_zscore,songtitle_metalness_mean_zscore,songtitle_metalness_sum_div_len2_zscore,bandname_metalness_sum_zscore,bandname_metalness_mean_zscore,bandname_metalness_sum_div_len2_zscore
num_songs_zscore,1.000000,-0.063847,-0.035927,0.038733,-0.030990,0.019101,0.022468,0.023494,0.449803,0.068983,0.153409,0.020011,0.020656,0.020677
release_year_zscore,-0.063847,1.000000,-0.196096,-0.312902,-0.231317,-0.042594,-0.038621,-0.041059,-0.084655,-0.045006,-0.056918,-0.019313,-0.029553,-0.026162
review_score_num_zscore,-0.035927,-0.196096,1.000000,0.203497,0.521398,0.004268,-0.001382,-0.000846,-0.046577,-0.022281,-0.028331,0.008874,0.009939,0.009673
user_review_num_zscore,0.038733,-0.312902,0.203497,1.000000,0.253465,0.016729,0.012479,0.014715,0.063567,0.032759,0.041563,0.023605,0.048980,0.039552
user_review_score_num_zscore,-0.030990,-0.231317,0.521398,0.253465,1.000000,0.007253,0.008844,0.008200,0.001452,0.005876,0.007654,0.021357,0.037260,0.031936
album_metalness_sum_zscore,0.019101,-0.042594,0.004268,0.016729,0.007253,1.000000,0.877294,0.949025,0.298502,0.264370,0.296352,0.101768,0.094298,0.099653
album_metalness_mean_zscore,0.022468,-0.038621,-0.001382,0.012479,0.008844,0.877294,1.000000,0.982011,0.284075,0.277716,0.305646,0.092856,0.091964,0.094439
album_metalness_sum_div_len2_zscore,0.023494,-0.041059,-0.000846,0.014715,0.008200,0.949025,0.982011,1.000000,0.300355,0.282861,0.313540,0.099778,0.096181,0.099925
songtitle_metalness_sum_zscore,0.449803,-0.084655,-0.046577,0.063567,0.001452,0.298502,0.284075,0.300355,1.000000,0.640576,0.761088,0.104652,0.101689,0.105101
songtitle_metalness_mean_zscore,0.068983,-0.045006,-0.022281,0.032759,0.005876,0.264370,0.277716,0.282861,0.640576,1.000000,0.970993,0.094678,0.094620,0.096718


## Linear Regression

We're going to run linear regressions using both the editor review score and user review scores as our outcome measures, starting with the editor review scores. For the editor review score model we will need to drop rows that are missing editor review scores.

### Regression -- Editor review scores

In [9]:
model_df1 = model_df.dropna(subset = ['review_score_num_zscore'])

X1 = model_df1[['num_songs_zscore', 'release_year_zscore', 'user_review_num_zscore', 'album_metalness_sum_zscore',
             'album_metalness_mean_zscore', 'album_metalness_sum_div_len2_zscore', 'songtitle_metalness_sum_zscore',
             'songtitle_metalness_mean_zscore', 'songtitle_metalness_sum_div_len2_zscore',
             'bandname_metalness_sum_zscore', 'bandname_metalness_mean_zscore', 'bandname_metalness_sum_div_len2_zscore']]
y1 = model_df1['review_score_num_zscore']

X1a = sm.add_constant(X1)
est1 = sm.OLS(y1, X1a)
results1 = est1.fit()
print(results1.summary())

                               OLS Regression Results                              
Dep. Variable:     review_score_num_zscore   R-squared:                       0.067
Model:                                 OLS   Adj. R-squared:                  0.066
Method:                      Least Squares   F-statistic:                     40.78
Date:                     Mon, 15 Oct 2018   Prob (F-statistic):           1.27e-93
Time:                             22:30:40   Log-Likelihood:                -9260.9
No. Observations:                     6805   AIC:                         1.855e+04
Df Residuals:                         6792   BIC:                         1.864e+04
Df Model:                               12                                         
Covariance Type:                 nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

Well that didn't work very well. Let's see if we only use one set of our metalness scores, since the different
sets of metalness scores are highly intercorrelated. The album's mean metalness score had the highest coefficient, so let's try all of the mean scores.

In [10]:
X2 = model_df1[['num_songs_zscore', 'release_year_zscore', 'user_review_num_zscore',
                'album_metalness_mean_zscore', 'songtitle_metalness_mean_zscore',
                'bandname_metalness_mean_zscore']]
y2 = model_df1['review_score_num_zscore']

X2a = sm.add_constant(X2)
est2 = sm.OLS(y2, X2a)
results2 = est2.fit()
print(results2.summary())

                               OLS Regression Results                              
Dep. Variable:     review_score_num_zscore   R-squared:                       0.064
Model:                                 OLS   Adj. R-squared:                  0.063
Method:                      Least Squares   F-statistic:                     77.76
Date:                     Mon, 15 Oct 2018   Prob (F-statistic):           2.46e-94
Time:                             22:30:41   Log-Likelihood:                -9271.8
No. Observations:                     6805   AIC:                         1.856e+04
Df Residuals:                         6798   BIC:                         1.861e+04
Df Model:                                6                                         
Covariance Type:                 nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

### Regression -- User review scores

For comparison, let's run a regression using user review scores as our output variable:

In [11]:
X3 = model_df[['num_songs_zscore', 'release_year_zscore', 'user_review_num_zscore', 'album_metalness_sum_zscore',
             'album_metalness_mean_zscore', 'album_metalness_sum_div_len2_zscore', 'songtitle_metalness_sum_zscore',
             'songtitle_metalness_mean_zscore', 'songtitle_metalness_sum_div_len2_zscore',
             'bandname_metalness_sum_zscore', 'bandname_metalness_mean_zscore', 
             'bandname_metalness_sum_div_len2_zscore']]
y3 = model_df['user_review_score_num_zscore']

X3a = sm.add_constant(X3)
est3 = sm.OLS(y3, X3a)
results3 = est3.fit()
print(results3.summary())


                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.094
Model:                                      OLS   Adj. R-squared:                  0.092
Method:                           Least Squares   F-statistic:                     63.40
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):          2.55e-147
Time:                                  22:30:42   Log-Likelihood:                -10085.
No. Observations:                          7363   AIC:                         2.020e+04
Df Residuals:                              7350   BIC:                         2.028e+04
Df Model:                                    12                                         
Covariance Type:                      nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--

Let's try a subset of metalness scores again. This time it's our sum divided by half the length score that has the best coefficient (for the band name this time), so let's try that set.

In [12]:
X4 = model_df[['num_songs_zscore', 'release_year_zscore', 'user_review_num_zscore',
                'album_metalness_sum_div_len2_zscore', 'songtitle_metalness_sum_div_len2_zscore',
                'bandname_metalness_sum_div_len2_zscore']]
y4 = model_df['user_review_score_num_zscore']

X4a = sm.add_constant(X4)
est4 = sm.OLS(y4, X4a)
results4 = est4.fit()
print(results4.summary())

                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.093
Model:                                      OLS   Adj. R-squared:                  0.092
Method:                           Least Squares   F-statistic:                     125.4
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):          1.78e-151
Time:                                  22:30:43   Log-Likelihood:                -10089.
No. Observations:                          7363   AIC:                         2.019e+04
Df Residuals:                              7356   BIC:                         2.024e+04
Df Model:                                     6                                         
Covariance Type:                      nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--

This isn't looking particularly promising, although at least the number of user reviews seems to have a consistently decent coefficiant (at least compared to everything else). 

However, just for fun let's run one using only one set of metalness scores.

### Regression -- User review scores using only metalness scores

In [13]:
X5 = model_df[['album_metalness_sum_div_len2_zscore', 'songtitle_metalness_sum_div_len2_zscore',
             'bandname_metalness_sum_div_len2_zscore']]
y5 = model_df['user_review_score_num_zscore']

X5a = sm.add_constant(X5)
est5 = sm.OLS(y5, X5a)
results5 = est5.fit()
print(results5.summary())

                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.001
Model:                                      OLS   Adj. R-squared:                  0.001
Method:                           Least Squares   F-statistic:                     2.585
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):             0.0514
Time:                                  22:30:44   Log-Likelihood:                -10443.
No. Observations:                          7363   AIC:                         2.089e+04
Df Residuals:                              7359   BIC:                         2.092e+04
Df Model:                                     3                                         
Covariance Type:                      nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--

### Regression -- User review scores using only other album metadata

It's pretty clear that the metalness scores have no predictive value for either of our prospective outcome variables.

However, the above regressions show two slightly interesting results: the album year is negatively correlated with rating (meaning older albums were rated higher), and the number of user reviews is correlated with both user and editor ratings. The latter seems not to be particularly predictive, since it would make sense if better albums attract more users to the album's page to share their opinion. The former may indicate that site editors became slightly harsher raters over the course of the site's existence.

Just for fun let's run the model with just these two predictors, separately and then together.

In [14]:
X6 = model_df[['release_year_zscore']]
y6 = model_df['user_review_score_num_zscore']

X6a = sm.add_constant(X6)
est6 = sm.OLS(y6, X6a)
results6 = est6.fit()
print(results6.summary())

                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.054
Model:                                      OLS   Adj. R-squared:                  0.053
Method:                           Least Squares   F-statistic:                     416.1
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):           5.04e-90
Time:                                  22:30:45   Log-Likelihood:                -10245.
No. Observations:                          7363   AIC:                         2.049e+04
Df Residuals:                              7361   BIC:                         2.051e+04
Df Model:                                     1                                         
Covariance Type:                      nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------

In [15]:
X7 = model_df[['user_review_num_zscore']]
y7 = model_df['user_review_score_num_zscore']

X7a = sm.add_constant(X7)
est7 = sm.OLS(y7, X7a)
results7 = est7.fit()
print(results7.summary())

                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.064
Model:                                      OLS   Adj. R-squared:                  0.064
Method:                           Least Squares   F-statistic:                     505.4
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):          2.67e-108
Time:                                  22:30:46   Log-Likelihood:                -10203.
No. Observations:                          7363   AIC:                         2.041e+04
Df Residuals:                              7361   BIC:                         2.042e+04
Df Model:                                     1                                         
Covariance Type:                      nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------

In [16]:
X8 = model_df[['release_year_zscore', 'user_review_num_zscore']]
y8 = model_df['user_review_score_num_zscore']

X8a = sm.add_constant(X8)
est8 = sm.OLS(y8, X8a)
results8 = est8.fit()
print(results8.summary())

                                 OLS Regression Results                                 
Dep. Variable:     user_review_score_num_zscore   R-squared:                       0.090
Model:                                      OLS   Adj. R-squared:                  0.090
Method:                           Least Squares   F-statistic:                     363.3
Date:                          Mon, 15 Oct 2018   Prob (F-statistic):          3.32e-151
Time:                                  22:30:47   Log-Likelihood:                -10101.
No. Observations:                          7363   AIC:                         2.021e+04
Df Residuals:                              7360   BIC:                         2.023e+04
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------

It sure looks like these two variables are much better predictors of album ratings than any of our metalness scores. Too bad! But I think that in and of itself is a pretty interesting finding.

Let's visualize how these two predictors relate to the average user review score -- but we will go back to the non-standardized variables, which we will do in another notebook.